In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
!pip install --quiet transformers==4.18.0
!pip install --quiet tokenizers==0.12.1
!pip install --quiet sentencepiece
!pip install --quiet japanize-matplotlib
!pip install transformers fugashi ipadic >> /dev/null

In [ ]:
! pip install git+https://github.com/rinnakk/japanese-clip.git

In [ ]:
! conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
! conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

import torch
import transformers

from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import sys
import os
import re
import random

from time import time
from tqdm import tqdm

from contextlib import contextmanager
import lightgbm as lgb

from pathlib import Path

import re
import requests
import unicodedata
import nltk
from nltk.corpus import wordnet
from bs4 import BeautifulSoup
nltk.download(['wordnet', 'stopwords', 'punkt'])

import japanese_clip as ja_clip
from torchvision.io import read_image

import cv2

In [ ]:
#上限表示数を拡張
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)

In [ ]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything(42)

In [ ]:
INPUT = "/content/drive/MyDrive/nishika" # 所望のディレクトリに変更してください。
train_image_path ="/content/drive/MyDrive/nishika/train/"
test_image_path ="/content/drive/MyDrive/nishika/test/"

In [ ]:
train_df = pd.read_csv(os.path.join(INPUT, "train.csv"))
test_df = pd.read_csv(os.path.join(INPUT, "test.csv"))
submission_df = pd.read_csv(os.path.join(INPUT, "sample_submission.csv"))

train_df["img_path"] = train_image_path + train_df["odai_photo_file_name"]
test_df["img_path"] = test_image_path + test_df["odai_photo_file_name"]

In [ ]:
import torchvision.models as models


from torch.utils.data import Dataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#import clip

model, preprocess = ja_clip.load("rinna/japanese-clip-vit-b-16", cache_dir="/tmp/japanese_clip", device=device)
tokenizer = ja_clip.load_tokenizer()

In [ ]:
def _get_img_paths(img_dir):
    img_dir = Path(img_dir)
    img_extensions = [".jpg"]
    img_paths = [str(p) for p in img_dir.iterdir() if p.suffix in img_extensions]
    img_paths.sort()

    return img_paths


class ImageFolder(Dataset):
    def __init__(self, img_dir):
        # 画像ファイルのパス一覧を取得する。
        self.img_paths = _get_img_paths(img_dir)
        #self.transform = transform

    def __getitem__(self, index):
        path = self.img_paths[index]
        img = preprocess(Image.open(path).convert("RGB"))
        #inputs = self.transform(img)

        return {"image": img, "path": path}

    def __len__(self):
        return len(self.img_paths)

In [ ]:
# Dataset を作成する。
dstrain = ImageFolder("/content/drive/MyDrive/nishika/train")

In [ ]:
import glob
import itertools


def get_images_features(dataset):
    image_features = []
    
    with torch.no_grad():
        for batch in tqdm(DataLoader(dataset, batch_size=8)):
            inputs = batch["image"].to(device)
            outputs = model.get_image_features(inputs)

            image_features.append(outputs)

    return torch.cat(image_features).cpu().numpy()

In [ ]:
train_image_features = get_images_features(dstrain)

100%|██████████| 1469/1469 [03:55<00:00,  6.23it/s]


In [ ]:
image_feature = pd.DataFrame.from_dict(train_image_features, orient='columns').add_prefix("clip_image_").reset_index()
image_feature.rename(columns={"columns":"odai_photo_file_name"}, inplace=True)

In [ ]:
image_feature = image_feature.drop(["index"], axis=1)

In [ ]:
# trainのデータに結合します。
train_df = pd.concat([train_df, image_feature],axis=1)

In [ ]:
train_df.shape

In [ ]:
# Dataset を作成する。
dstest = ImageFolder("/content/drive/MyDrive/nishika/test")

In [ ]:
test_image_features = get_images_features(dstest)

100%|██████████| 353/353 [00:56<00:00,  6.30it/s]


In [ ]:
image_feature = pd.DataFrame.from_dict(test_image_features, orient='columns').add_prefix("clip_image_").reset_index()
image_feature.rename(columns={"columns":"odai_photo_file_name"}, inplace=True)

In [ ]:
print(image_feature)

In [ ]:
image_feature = image_feature.drop(["index"], axis=1)

In [ ]:
print(image_feature)

In [ ]:
# testのデータに結合します。
test_df = pd.concat([test_df, image_feature],axis=1)

In [ ]:
test_df.shape

In [ ]:
## テキストの欠損値を補間します
train_df["text"] = train_df["text"].fillna('NaN')
test_df["text"] = test_df["text"].fillna('NaN')

In [ ]:
import tensorflow as tf

def get_texts_features(dataset):
    text_labels = []
    
    with torch.no_grad():
        for labels in tqdm(DataLoader(dataset, batch_size=8)):
            inputs = ja_clip.tokenize(texts = labels, device = device)
            outputs = model.get_text_features(**inputs)
            text_labels.append(outputs)

    return torch.cat(text_labels).cpu().numpy()

In [ ]:
train_labels = get_texts_features(train_df["text"])

100%|██████████| 368/368 [07:09<00:00,  1.17s/it]


In [ ]:
test_labels = get_texts_features(test_df["text"])

100%|██████████| 89/89 [01:43<00:00,  1.17s/it]


In [ ]:
print(train_labels)

In [ ]:
## テキスト特徴量
features_text_train_df = pd.DataFrame(train_labels).add_prefix("clip-text")
features_text_test_df = pd.DataFrame(test_labels).add_prefix("clip-text")

train_df = pd.concat([train_df, features_text_train_df], axis=1)
test_df = pd.concat([test_df, features_text_test_df], axis=1)

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.to_csv('/content/drive/MyDrive/nishika/embeded/embedding_train_rinna_clip.csv')

In [ ]:
test_df.to_csv('/content/drive/MyDrive/nishika/embeded/embedding_test_rinna_clip.csv')